# Mes imports 

In [11]:

import sys
sys.path.append('../')
import numpy as np
import pymongo
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.sparse import csr_matrix
import os
import mlflow
import warnings
warnings.filterwarnings("ignore")

### Connexion à la db

In [12]:
# Je me connecte a ma database mongo 
client = pymongo.MongoClient('localhost:27017')
db = client['movie']
movies = db['movie_json']
users = db['user_json']

### Construire une liste à plat: user / movieid / rating / timestamp

In [13]:
# Extraire les données de la collection
data = list(users.find())

# Créer une liste vide pour chaque colonne du DataFrame
user_list = []
movieid_list = []
rating_list = []
timestamp_list = []

# Parcourir les données et extraire les informations nécessaires
for entry in data:
    user_id = entry['_id']
    for movie in entry['movies']:
        user_list.append(user_id)
        movieid_list.append(movie['movieid'])
        rating_list.append(movie['rating'])
        timestamp_list.append(movie['timestamp'])

# Créer le DataFrame
df2 = pd.DataFrame({
    'user': user_list,
    'movieid': movieid_list,
    'rating': rating_list,
    'timestamp': timestamp_list
})

# Convertir le timestamp en datetime
df2['timestamp'] = pd.to_datetime(df2['timestamp'], unit='s')

# Afficher les premières lignes du DataFrame
print(df2.head())


   user  movieid  rating           timestamp
0  6040      573       4 2000-04-25 23:07:36
1  6040      589       4 2000-04-25 23:23:16
2  6040        1       3 2000-05-07 16:35:58
3  6040     2068       4 2001-08-10 14:33:02
4  6040      592       2 2000-04-26 02:26:56


In [14]:
print(df2.shape)
print(df2.min())

(1000209, 4)
user                           1
movieid                        1
rating                         1
timestamp    2000-04-25 23:05:32
dtype: object


In [15]:
df_sorted = df2.sort_values(by='timestamp')
print(df_sorted.head())

     user  movieid  rating           timestamp
270  6040      858       4 2000-04-25 23:05:32
285  6040     2384       4 2000-04-25 23:05:54
5    6040      593       5 2000-04-25 23:05:54
324  6040     2019       5 2000-04-25 23:06:17
139  6040     1961       4 2000-04-25 23:06:17


In [16]:
def partition(df, test_size=0.2, mini_size=0.03):
    # Partition the dataset into a training set and a test set
    train_data, test_data = train_test_split(df, test_size=test_size, random_state=42)

    # Reduce the size of the training set and the test set for better performance
    train_mini = train_data[:int(mini_size * len(train_data))]
    test_mini = test_data[:int(mini_size * len(test_data))]

    return train_data, test_data, train_mini, test_mini

In [17]:
def partition_time(df,mini_size=0.8):
    df_sorted = df.sort_values(by='timestamp')
    train_size = int(len(df_sorted) * mini_size)
    train_df = df_sorted[:train_size]
    test_df = df_sorted[train_size:]
    return train_df, test_df

### Splitter les données en test et train

In [18]:
# # Calculate the index where 80% of the data ends
# train_size = int(0.8 * len(df2))

# # Split the dataframe
# df_train = df_sorted[:train_size]
# df_test = df_sorted[train_size:]

# # train_df will contain 80% of the data based on timestamp for training
# # test_df will contain 20% of the data based on timestamp for testing
# print(df_train.head())
# print(df_test.head())

In [19]:
train_size = 0.8
split_index = int(train_size * len(df2))
# Split the data into train and test sets
train_df = df_sorted[:226331]
test_df = df_sorted[226331:256331]
print(train_df.head())
print(test_df.head())

     user  movieid  rating           timestamp
270  6040      858       4 2000-04-25 23:05:32
285  6040     2384       4 2000-04-25 23:05:54
5    6040      593       5 2000-04-25 23:05:54
324  6040     2019       5 2000-04-25 23:06:17
139  6040     1961       4 2000-04-25 23:06:17
        user  movieid  rating           timestamp
248292  4482      627       4 2000-08-01 03:39:16
258716  4430     2686       4 2000-08-01 03:39:24
258774  4430      150       2 2000-08-01 03:39:24
258654  4430       45       2 2000-08-01 03:39:24
258745  4430     2329       2 2000-08-01 03:39:24


In [20]:
# Appeler la fonction partition avec votre DataFrame
train_data, test_data, train_mini, test_mini = partition(df2, test_size=0.2, mini_size=0.03)

# Utiliser les ensembles de données retournés
# Par exemple, pour accéder aux données d'entraînement initiales :
print("Taille de l'ensemble d'entraînement initial :", len(train_data))
print("Taille de l'ensemble de test initial :", len(test_data))

# Pour accéder aux sous-ensembles réduits :
print("Taille du sous-ensemble d'entraînement réduit :", len(train_mini))
print("Taille du sous-ensemble de test réduit :", len(test_mini))

Taille de l'ensemble d'entraînement initial : 800167
Taille de l'ensemble de test initial : 200042
Taille du sous-ensemble d'entraînement réduit : 24005
Taille du sous-ensemble de test réduit : 6001


In [23]:
# Créer un ensemble d'utilisateurs pour chaque DataFrame
users_train = set(train_df['user'])
users_test = set(test_df['user'])

# Trouver les utilisateurs communs
users_common = users_train.intersection(users_test)

# Calculer les utilisateurs dans le train mais pas dans le test
users_train_not_in_test = users_train - users_test
num_users_train_not_in_test = len(users_train_not_in_test)

# Calculer les utilisateurs dans le test mais pas dans le train
users_test_not_in_train = users_test - users_train
num_users_test_not_in_train = len(users_test_not_in_train)

print("Nombre d'utilisateurs dans le train mais pas dans le test :", num_users_train_not_in_test)
print("Nombre d'utilisateurs dans le test mais pas dans le train :", num_users_test_not_in_train)

Nombre d'utilisateurs dans le train mais pas dans le test : 1560
Nombre d'utilisateurs dans le test mais pas dans le train : 218


In [26]:
# Convertir l'ensemble en liste si nécessaire
users_common_list = list(users_common)
print("Nombre d'utilisateurs communs aux deux DataFrames :", len(users_common))

# Filtrer les lignes de test_df pour lesquelles 'user' est dans users_common_list
test_df_filtered_user = df_test[df_test['user'].isin(users_common_list)]

# Filtrer les lignes de df_test pour lesquelles 'movies' est dans users_common_list
df_test_filtered = df_test_filtered_user[df_test_filtered_user['movieid'].isin(movies_common_list)]

# Afficher le DataFrame filtré
print(test_df)
print(df_test_filtered)

Nombre d'utilisateurs communs aux deux DataFrames : 50


NameError: name 'df_test' is not defined

In [25]:
# Convertir l'ensemble en liste si nécessaire
users_common = users_train.intersection(users_test)
users_common_list = list(users_common)
print("Nombre d'utilisateurs communs aux deux DataFrames :", len(users_common))

# Filtrer les lignes de test_df pour lesquelles 'user' est dans users_common_list
test_df_filtered_user = test_df[test_df['user'].isin(users_common_list)]

# Filtrer les lignes de test_df pour lesquelles 'movies' est dans users_common_list
test_df_filtered = test_df_filtered_user[test_df_filtered_user['movieid'].isin(movies_common_list)]

# Afficher le DataFrame filtré
print(test_df)
print(test_df_filtered)

Nombre d'utilisateurs communs aux deux DataFrames : 50
        user  movieid  rating           timestamp
248292  4482      627       4 2000-08-01 03:39:16
258716  4430     2686       4 2000-08-01 03:39:24
258774  4430      150       2 2000-08-01 03:39:24
258654  4430       45       2 2000-08-01 03:39:24
258745  4430     2329       2 2000-08-01 03:39:24
...      ...      ...     ...                 ...
295654  4224     1801       4 2000-08-03 16:04:23
295776  4224       74       3 2000-08-03 16:04:23
295713  4224     1550       3 2000-08-03 16:04:23
295656  4224      207       2 2000-08-03 16:04:23
295869  4224      140       3 2000-08-03 16:04:23

[30000 rows x 4 columns]
        user  movieid  rating           timestamp
248292  4482      627       4 2000-08-01 03:39:16
258716  4430     2686       4 2000-08-01 03:39:24
258774  4430      150       2 2000-08-01 03:39:24
258654  4430       45       2 2000-08-01 03:39:24
258745  4430     2329       2 2000-08-01 03:39:24
...      ...      .

In [ ]:
# Appeler la fonction partition_time avec votre DataFrame
train_df, test_df = partition_time(df2, mini_size=0.8)

# Utiliser les ensembles de données retournés
# Par exemple, pour accéder aux données d'entraînement :
print("Taille de l'ensemble d'entraînement :", len(train_df))

# Pour accéder aux données de test :
print("Taille de l'ensemble de test :", len(test_df))

In [ ]:
df_train.shape, df_test.shape

### utilisateurs en commun 

### Films en commun

In [24]:
# Créer un ensemble de movies pour chaque DataFrame
movies_train = set(train_df['movieid'])
movies_test = set(test_df['movieid'])

# Trouver les movies communs
movies_common = movies_train.intersection(movies_test)

# Convertir l'ensemble en liste si nécessaire
movies_common_list = list(movies_common)
print("Nombre de films commun aux deux DataFrames :", len(movies_common))

# Calculer les utilisateurs dans le train mais pas dans le test
movies_train_not_in_test = movies_train - movies_test
num_movies_train_not_in_test = len(movies_train_not_in_test)

# Calculer les utilisateurs dans le test mais pas dans le train
movies_train_not_in_train = movies_test - movies_train
num_movies_test_not_in_train = len(movies_train_not_in_train)

print("Nombre de films dans le train mais pas dans le test :", num_movies_train_not_in_test)
print("Nombre de films  dans le test mais pas dans le train :", num_movies_test_not_in_train)

Nombre de films commun aux deux DataFrames : 2681
Nombre de films dans le train mais pas dans le test : 641
Nombre de films  dans le test mais pas dans le train : 26


### Clean des données (user et movie) de TEST qui ne sont pas inclus dans train

### Trouver l'utilisateur commun qui a vu le + de film dans le df_test

In [ ]:
films_vus_par_utilisateur = df_test_filtered.groupby('user')['movieid'].count()

# Filtrer les films vus par les utilisateurs communs
films_vus_par_utilisateur_common = films_vus_par_utilisateur[users_common_list]

# Trouver l'utilisateur avec le plus grand nombre de films vus
utilisateur_max_films_vus = films_vus_par_utilisateur_common.idxmax()

# Afficher l'utilisateur avec le plus grand nombre de films vus
print("L'utilisateur avec le plus grand nombre de films vus dans df_test est :", utilisateur_max_films_vus)

In [ ]:
# Filtrer les lignes où la colonne 'user' est égale à 1088
df_test_user_1088 = df_test.loc[df_test['user'] == 1088]

# Afficher les lignes filtrées
print(df_test_user_1088)

In [ ]:
# Filtrer les lignes où la colonne 'user' est égale à 1680 dans la base de test
df_train_user_1080 = df_train[df_train['user'] == 1680]

# Compter le nombre de lignes (films) dans le dataframe filtré
nombre_films_vus_par_user_1080 = len(df_train_user_1080)

print("Nombre de films vus par l'utilisateur 1080 dans la base de train :", nombre_films_vus_par_user_1080)

In [ ]:
user_to_check = 1088
if user_to_check in users_common_list:
    print("L'utilisateur", user_to_check, "est dans la liste des utilisateurs communs.")
else:
    print("L'utilisateur", user_to_check, "n'est pas dans la liste des utilisateurs communs.")

### Trouver l'utilisateur commun qui a vu le + de film dans le df_train

In [ ]:
# Calculer le nombre de films vus par utilisateur dans le dataframe d'entraînement réel
films_vus_par_utilisateur_train = df_train.groupby('user')['movieid'].count()

# Filtrer les films vus par les utilisateurs communs
films_vus_par_utilisateur_common_train = films_vus_par_utilisateur_train[users_common_list]

# Trouver l'utilisateur avec le plus grand nombre de films vus dans l'ensemble d'entraînement réel
utilisateur_max_films_vus_train = films_vus_par_utilisateur_common_train.idxmax()

# Afficher l'utilisateur avec le plus grand nombre de films vus dans l'ensemble d'entraînement réel
print("L'utilisateur avec le plus grand nombre de films vus dans df_train est :", utilisateur_max_films_vus_train)


In [ ]:
# Filtrer les lignes où la colonne 'user' est égale à 1680 dans la base d'entraînement réelle
df_train_user_1680 = df_train.loc[df_train['user'] == 1680]

# Afficher les lignes filtrées
print(df_train_user_1680)


In [ ]:
user_to_check = 1680
if user_to_check in users_common_list:
    print("L'utilisateur", user_to_check, "est dans la liste des utilisateurs communs.")
else:
    print("L'utilisateur", user_to_check, "n'est pas dans la liste des utilisateurs communs.")

In [ ]:
# Filtrer les lignes où la colonne 'user' est égale à 1680 dans la base de test
df_test_user_1680 = df_test[df_test['user'] == 1680]

# Compter le nombre de lignes (films) dans le dataframe filtré
nombre_films_vus_par_user_1680 = len(df_test_user_1680)

# Afficher le nombre de films vus par l'utilisateur 1680 dans la base de test
print("Nombre de films vus par l'utilisateur 1680 dans la base de test :", nombre_films_vus_par_user_1680)


### Obtenir une matrice de rating réel

In [ ]:
df_train_matrix = pd.pivot_table(df_train, index='user', columns= 'movieid', values = 'rating', fill_value=0)

#suppression des user qui auraient 0 notes
df_train_matrix = df_train_matrix[df_train_matrix.sum(axis=1) > 0]
df_train_matrix

In [ ]:
# Calcul du nombre total d'éléments
total_elements = df_train_matrix.size

# Calcul du nombre de zéros
nombre_zeros = np.count_nonzero(df_train_matrix == 0)

# Calcul de la sparsité
sparsité = (nombre_zeros / total_elements) * 100

print("nombre de zeros : ", nombre_zeros)
print("total d'elements : ", total_elements)
print("Sparsité de la matrice : {:.0f}%".format(sparsité))

In [ ]:
df_sparse = df_train_matrix.astype(pd.SparseDtype("float64", 0))
df_sparse

In [ ]:
component = 50
iteration = 200

params = {
    'n_components' : component,
    'max_iter' : iteration
}

# Entrainement du modèle
nmf = NMF(**params)
nmf.fit(df_sparse)

# Faire des prédictions sur les données de test 
U = nmf.transform(df_sparse)
M = nmf.components_ #this are the movies and can not change from the database predict to the database test or transform the perimeter has to be the same as the fit datatbase
pred_matrix = np.dot(U, M) #this is giving for each user a rating for ALL the movies
# we transform the pred_matrix into a dataframe in columns to make sure we can compare training database with the predict results
pred_matrix = pd.DataFrame(pred_matrix, index=df_sparse.index, columns=df_sparse.columns)
pred_matrix

In [ ]:
# # Normalisation des prédictions
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaled_predictions = scaler.fit_transform(pred_matrix)
# scaled_predictions_df = pd.DataFrame(scaled_predictions, index=df_sparse.index, columns=df_sparse.columns)

In [ ]:
# Comparaison avec les données d'entraînement
# df_depivoted = scaled_predictions_df.stack().reset_index()
df_depivoted = pred_matrix.stack().reset_index()
df_depivoted.columns = ['user', 'movieid', 'predict']
df_depivoted

In [ ]:
df_train
df_train.describe()

In [ ]:
# Intégration avec les données d'entraînement
df_compare = pd.merge(df_depivoted,df_train, how='inner', on=['user', 'movieid'])
df_compare['delta'] = df_compare['rating'] / df_compare['predict']
df_compare 

In [ ]:
# calcule la moyenne des écarts au carré entre les vraies valeurs de notation et les valeurs prédites
mse = mean_squared_error(df_compare['rating'], df_compare['predict'])
# mesure de l'erreur moyenne entre les valeurs prédites et les valeurs réelles
delta_mse = np.sqrt(mean_squared_error(df_compare['rating'], df_compare['predict']))
print('delta mse train: ', delta_mse)
print('mse train : ',mse)

### Calcul du top 10

In [ ]:
df_predict_notseen_movies = df_depivoted.drop(df_compare.index)
df_predict_notseen_movies.describe()

In [ ]:
df_predict_notseen_movies

In [ ]:
# Join unpivoted predictions with the original train_df
df_pred_vs_test = df_predict_notseen_movies.merge(df_test_filtered, on=['user', 'movieid'])
df_pred_vs_test

In [ ]:
# Sorting my df_pred by ascending users and descending scores
df_pred_vs_test.sort_values(by=['user', 'predict'], ascending=[True, False], inplace=True)
df_pred_vs_test

In [ ]:
df_pred_vs_test.describe()

In [ ]:
# Filtering to only keep the 10 best scores for each user
top_10_per_user = df_pred_vs_test.sort_values(by=['user', 'predict'], ascending=[True, False]) \
                         .groupby('user').head(10)

top_10_per_user

In [ ]:
# Using my top 10 movies to check my model performance
average_rating = top_10_per_user['rating'].mean()
diff_rating = 5 - average_rating
print("Average rating:", average_rating)
print("Difference from 5:", diff_rating)

In [ ]:
# Filtrage pour ne conserver que les 10 pires scores pour chaque utilisateur
bad_10_per_user = df_pred_vs_test.sort_values(by=['user', 'predict'], ascending=[True, False]) \
                                  .groupby('user').tail(10)

bad_10_per_user

In [ ]:
# Using my top 10 movies to check my model performance
average_rating_bad = bad_10_per_user['rating'].mean()
diff_rating_bad =  average_rating_bad
print("Average rating:", average_rating_bad)
print("Difference from 2:", diff_rating_bad)

In [ ]:
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("delta mse", delta_mse)
    mlflow.log_metric("average_rating", average_rating)
    mlflow.log_metric("rating_diff", diff_rating)
    mlflow.log_metric("average_rating_bad", average_rating_bad)
    mlflow.log_metric("rating_diff_bad", diff_rating_bad)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("On training datas", f"On the training database, components {component}, iteration {iteration} ")

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=NMF,
        artifact_path="NMF_Model",
        signature=False,
        input_example=df_train_matrix,
        registered_model_name=f"NMF_on_train_movies_users {component} components and {iteration} iteration max",
    )
